In [2]:
import pandas as pd
import numpy as np

This script downloads the file containing trigger data, which includes the moments each scene starts, the type of each scene, and when the button was pressed. Some triggers may fail due to unexpected technical issues; in these instances, we assign zero to these particular events. Additionally, if subjects do not press the button, we also assign zero to these cases.

Using this trigger data, we perform the following steps:

1) Remove all events where the scene start trigger failed.
2) Remove all events where the button trigger did not work.
3) Remove control scenes where the age and gender of the pedestrians remain the same on both sides.
4) Calculate the response time for each scene.
5) Separate picture-based scenes from video-based scenes.
6) Remove all scenes with abnormally long or short response times using the 95% confidence interval.
7) Create protocols for each subject containing the moments when the scene starts and when the button is pressed.
8) Create protocols for each subject containing the information about the trial to extract from EEG data

# Download file with trigger data

In [3]:
data1=pd.read_csv('TRIGGERS.csv')
data1 = data1[(data1['SS Latency']> 1)].copy()              # remove scenes where the scene-start (SS) event is not correctly determined 
data1 = data1[(data1['BP Latency']> 1)].copy()              # remove scenes where the button-press (BP) event is not correctly determined
data1 = data1[(data1['Type']!= 'control')].copy()           # remove control scenes
data1['RT']=(data1['BP Latency']-data1['SS Latency'])/300   # add response time to each scene

# Check RT for Picture-based scenes

In [4]:
d=data1[data1['Scene type']=='Text']
d=d[(d['RT']> 0)].copy()


q_low = d['RT'].quantile(0.05)
q_hi  = d['RT'].quantile(0.95)
d_text = d[(d['RT'] < q_hi) & (d['RT'] > q_low)] # remove very long and very short RTs


print(len(d_text['RT']))
print('median RT=',np.median(d_text['RT']))
print('25% RT=',np.percentile(d_text['RT'], 25))
print('75% RT=',np.percentile(d_text['RT'], 75))

1090
median RT= 3.581666666666667
25% RT= 2.65
75% RT= 5.13


# Check RT for Video-based scenes

In [6]:
d=data1[data1['Scene type']=='Video']
d=d[(d['RT']> 0)].copy()

q_low = d['RT'].quantile(0.01)
q_hi  = d['RT'].quantile(0.99)
d_video = d[(d['RT'] < q_hi) & (d['RT'] > q_low)] # remove very long and very short RTs

print(len(d_video['RT']))
print('median RT=',np.median(d_video['RT']))
print('25% RT=',np.percentile(d_video['RT'], 25))
print('75% RT=',np.percentile(d_video['RT'], 75))

951
median RT= 4.29
25% RT= 3.1950000000000003
75% RT= 5.31


# Make Video Protocols with RT for each subject

In [7]:
for participant in d_video['participant'].unique():
    protocol=d_video[d_video['participant']==participant][['SS Latency']]
    protocol['BP Latency']=d_video[d_video['participant']==participant][['BP Latency']]
    protocol['Type']=d_video[d_video['participant']==participant][['Type']]
    protocol.reset_index(drop=True, inplace=True)
    filename_SS='RT_protocol_'+str(participant)+'_'+'Video'+'.xlsx'
    protocol.to_excel(filename_SS,index=False)

# Make Video Protocols to extract EEG epochs for each subject

In [9]:
for participant in d_video['participant'].unique():
    protocol=d_video[d_video['participant']==participant][['SS Latency']]-1200 #left boundary 4s before scene starts
    protocol['SS Latency2']=protocol['SS Latency']+10*300+1200 #right boundary - 14s after scene start
    protocol['Shift']=0
    protocol.reset_index(drop=True, inplace=True)
    filename_SS='protocol_'+str(participant)+'_'+'Video'+'.xlsx'
    protocol.to_excel(filename_SS,index=False)

# Make Text Protocols with RT for each subject

In [10]:
for participant in d_text['participant'].unique():
    protocol=d_text[d_text['participant']==participant][['SS Latency']]
    protocol['BP Latency']=d_text[d_text['participant']==participant][['BP Latency']]
    protocol['Type']=d_text[d_text['participant']==participant][['Type']]
    protocol.reset_index(drop=True, inplace=True)
    filename_SS='RT_protocol_'+str(participant)+'_'+'Text'+'.xlsx'
    protocol.to_excel(filename_SS,index=False)

# Make Text Protocols to extract EEG epochs for each subject

In [11]:
for participant in d_text['participant'].unique():
    protocol=d_text[d_text['participant']==participant][['SS Latency']]-1200 #left boundary 4s before scene starts
    protocol['SS Latency2']=protocol['SS Latency']+10*300+1200 #right boundary - 14s after scene start
    protocol['Shift']=0
    protocol.reset_index(drop=True, inplace=True)
    filename_SS='protocol_'+str(participant)+'_'+'Text'+'.xlsx'
    protocol.to_excel(filename_SS,index=False)